In [37]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
df = pd.read_csv('../../dados/csv_result-ebay_confianca_completo.csv')
print(df.shape)
df.head()

(5806, 75)


,feedbacks,reviews,blacklist,mextraversion,cextraversion,sextraversion,mneuroticism,cneuroticism,sneuroticism,mconscientiousness,...,need_practicaly,need_selfexpression,need_stability,need_structure,value_conservation,value_openess,value_hedonism,value_selfenhancement,value_selftranscendence,reputation
2,0,49,N,4.181642,0.6,1,2.777591,0.6,0,4.085460,...,0.696359,0.698786,0.756963,0.660119,0.619416,0.746372,0.640073,0.598037,0.828716,Bom
3,0,56,N,4.007042,0.6,0,2.698650,0.6,0,4.187338,...,0.715300,0.664572,0.728806,0.660740,0.588969,0.735915,0.644465,0.603042,0.809379,Bom
4,0,50,N,4.538230,0.7,1,2.298492,0.5,1,5.085833,...,0.720150,0.694678,0.669652,0.627962,0.553523,0.766618,0.655470,0.645042,0.826039,Bom
5,72,0,N,4.692854,0.3,0,2.987231,0.5,0,4.831320,...,0.739793,0.637027,0.697221,0.638587,0.675289,0.752234,0.679661,0.674438,0.813391,Bom
6,76,0,N,4.966753,0.3,0,3.048730,0.5,0,4.725294,...,0.718530,0.616852,0.692761,0.646695,0.677245,0.699785,0.648607,0.616075,0.816841,Bom


In [5]:
df['reputation'].value_counts()

Bom     4299
Ruim    1507
Name: reputation, dtype: int64

In [6]:
4299 / 5806

0.7404409231829142

In [7]:
1507 / 5806

0.2595590768170858

In [8]:
df['blacklist'].value_counts()

N    5022
B     784
Name: blacklist, dtype: int64

In [9]:
troca = {'N' : 0, 'B' : 1}
df['blacklist'] = df['blacklist'].map(troca)

In [11]:
df['blacklist'].value_counts()

0    5022
1     784
Name: blacklist, dtype: int64

In [12]:
df['reputation'].value_counts()

Bom     4299
Ruim    1507
Name: reputation, dtype: int64

In [13]:
troca = {'Bom' : 0, 'Ruim' : 1}
df['reputation'] = df['reputation'].map(troca)

In [14]:
df['reputation'].value_counts()

0    4299
1    1507
Name: reputation, dtype: int64

In [19]:
df.isnull().sum().sum()

0

### Dados desbalanceados

In [28]:
X = df.iloc[:, 0:-1].values
X.shape

(5806, 74)

In [30]:
y = df.iloc[:, -1].values
y.shape

(5806,)

In [31]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = .2, stratify=y)

In [32]:
X_treino.shape, X_teste.shape

((4644, 74), (1162, 74))

In [33]:
y_treino.shape, y_teste.shape

((4644,), (1162,))

In [39]:
%%time
modelo = RandomForestClassifier(n_estimators=100)
modelo.fit(X_treino, y_treino)
previsoes = modelo.predict(X_teste)
accuracy_score(previsoes, y_teste)

CPU times: total: 2.98 s
Wall time: 2.99 s


0.7435456110154905

In [48]:
cm = confusion_matrix(previsoes, y_teste)
cm

array([[853, 291],
       [  7,  11]], dtype=int64)

In [51]:
853 / (853 + 291)

0.7456293706293706

In [52]:
11 / (11 + 7)

0.6111111111111112

### Subamostragem

In [40]:
from imblearn.under_sampling import NearMiss

In [41]:
nm = NearMiss(version=1)
X_under, y_under = nm.fit_sample(X,y)

In [42]:
np.unique(y, return_counts=True)

(array([0, 1], dtype=int64), array([4299, 1507], dtype=int64))

In [43]:
np.unique(y_under, return_counts=True)

(array([0, 1], dtype=int64), array([1507, 1507], dtype=int64))

In [44]:
X_treino_u, X_teste_u, y_treino_u, y_teste_u = train_test_split(X_under, y_under, test_size=0.2, stratify=y_under)

In [45]:
X_treino_u.shape, X_teste_u.shape

((2411, 74), (603, 74))

In [46]:
y_treino_u.shape, y_teste_u.shape

((2411,), (603,))

In [47]:
modelo_u = RandomForestClassifier()
modelo_u.fit(X_treino_u, y_treino_u)
previsoes_u = modelo_u.predict(X_teste_u)
accuracy_score(previsoes_u, y_teste_u)

0.7014925373134329

In [53]:
cm_u = confusion_matrix(previsoes_u, y_teste_u)
cm_u

array([[223, 102],
       [ 78, 200]], dtype=int64)

In [54]:
223 / (223 + 102)

0.6861538461538461

In [55]:
200 / (78 + 200)

0.7194244604316546

### Sobamostragem

In [56]:
from imblearn.over_sampling import SMOTE

In [57]:
smote = SMOTE()
X_over, y_over = smote.fit_sample(X,y)

In [58]:
X_over.shape, y_over.shape

((8598, 74), (8598,))

In [59]:
np.unique(y, return_counts=True)

(array([0, 1], dtype=int64), array([4299, 1507], dtype=int64))

In [60]:
np.unique(y_over, return_counts=True)

(array([0, 1], dtype=int64), array([4299, 4299], dtype=int64))

In [61]:
X_treino_o, X_teste_o, y_treino_o, y_teste_o = train_test_split(X_over, y_over, test_size=0.2, stratify=y_over)

In [62]:
X_treino_o.shape, X_teste_o.shape

((6878, 74), (1720, 74))

In [63]:
y_treino_o.shape, y_teste_o.shape

((6878,), (1720,))

In [65]:
modelo_o = RandomForestClassifier()
modelo_o.fit(X_treino_o, y_treino_o)
previsoes_o = modelo_o.predict(X_teste_o)
accuracy_score(previsoes_o, y_teste_o)

0.8302325581395349

In [66]:
cm_o = confusion_matrix(previsoes_o, y_teste_o)
cm_o

array([[721, 153],
       [139, 707]], dtype=int64)

In [67]:
721 / (721 + 153)

0.8249427917620137

In [68]:
707 / (707 + 139)

0.8356973995271868